In [1]:
import main

In [2]:
import numpy as np

In [3]:
precursor_1 = main.precursor_FDM(delay_group_num=1)
precursor_1.set_beta([0.0075])
precursor_1.set_lambda([0.08])

In [4]:
material_1 = main.material_FDM(mat_id=0)
material_1.set_XS('XS_scatter',[[0,0.01],[0,0]])
material_1.set_XS('Diffusion_coef',[1.4,0.4])
material_1.set_XS('XS_absorption',[0.01,0.15])
# material_1.set_XS('XS_absorption',[0.01,0.10392197])
material_1.set_XS('XS_nu_fission',[0.007,0.2])
material_1.set_XS('XS_fission_spectrum',[1.0,0.0])

In [5]:
material_2 = main.material_FDM(mat_id=1)
material_2.set_XS('XS_scatter',[[0,0.01],[0,0]])
material_2.set_XS('Diffusion_coef',[1.4,0.4])
material_2.set_XS('XS_absorption',[0.01,0.15])
# material_2.set_XS('XS_absorption',[0.02,0.15])
material_2.set_XS('XS_nu_fission',[0.007,0.2])
material_2.set_XS('XS_fission_spectrum',[1.0,0.0])

In [6]:
material_3 = main.material_FDM(mat_id=2)
material_3.set_XS('XS_scatter',[[0,0.01],[0,0]])
material_3.set_XS('Diffusion_coef',[1.3,0.5])
material_3.set_XS('XS_absorption',[0.008,0.05])
# material_3.set_XS('XS_absorption',[0.018,0.05])
material_3.set_XS('XS_nu_fission',[0.003,0.06])
material_3.set_XS('XS_fission_spectrum',[1.0,0.0])

In [7]:
geo = main.geometry_FDM(x_block=3,y_block=3)
sl = slice(0, 3, None)
geo.set_block_mat(material_3,sl,sl)
geo.set_block_mat(material_1,1,1)
geo.set_block_mat(material_2,1,0)
geo.set_block_mat(material_2,0,1)

geo.set_block_size(x_size=[24,32,24],y_size=[24,32,24])
geo.set_discretized_num(x_dim=[24,32,24],y_dim=[24,32,24])
geo.check_blocks()

Material layout:
[[2 1 2]
 [1 0 2]
 [2 2 2]]
Block size:
x:[24, 32, 24]
y:[24, 32, 24]
Discretization number:
x:[24, 32, 24]
y:[24, 32, 24]


In [8]:
case = main.solver_FDM(folder_name='TWIGL_Seed_Blanket_2D',group_num=2)
case.add_material([material_1,material_2,material_3])
case.set_geometry(geo,transient_mode=True)
case.set_precursor(precursor_1)
case.set_neutron_velocity([1e7,2e5])

In [9]:
# time_steps = np.array([0,0.005])
time_steps = np.linspace(0.0, 0.2, num=41)
# time_steps = np.array([0])
for time_index,time_step in enumerate(time_steps):
    print('Time={} (s)'.format(time_steps[time_index]))
    if time_index==0:
        # To get initial flux distribution
        case.solve_source_iter_correct(max_iter=50,k_tolerance=1e-5,flux_tolerance=1e-4,initial_k=1.0)
        case.initial_dynamics(transient_algorithm='Implicit_Euler')
        # case.initial_dynamics(transient_algorithm='SCM')
    else:
        # Update material
        m1_a2 = 0.15*(1-0.11667*time_step) # Linear 
        # m1_a2 = 0.15*0.97666 # Jump
        material_1.set_XS('XS_absorption',[0.01,m1_a2])
        case.update_material(0,material_1,mat_type='real')
        # Solve transient problem with SCM
        time_interval = time_steps[time_index]-time_steps[time_index-1]
        # case.solve_transient_SCM_2(time_index,time_interval,max_iter=100,k_tolerance=1e-5,flux_tolerance=1e-4,k_outer_tolerance=1e-7)
        case.solve_transient_Implicit_Euler(time_index,time_interval,max_iter=200,k_tolerance=1e-5,flux_tolerance=1e-4)

Time=0.0 (s)
####################################
Timestep 0 calculation begins
Iteration 1: Eigenvalue k = 0.844666951319332
Iteration 2: Eigenvalue k = 0.8899489163396996
Iteration 3: Eigenvalue k = 0.8994594776166439
Iteration 4: Eigenvalue k = 0.9038026479314232
Iteration 5: Eigenvalue k = 0.9064220494043872
Iteration 6: Eigenvalue k = 0.9081900679307711
Iteration 7: Eigenvalue k = 0.9094479494340278
Iteration 8: Eigenvalue k = 0.9103687896329565
Iteration 9: Eigenvalue k = 0.9110545506517272
Iteration 10: Eigenvalue k = 0.9115708863732308
Iteration 11: Eigenvalue k = 0.9119625156101635
Iteration 12: Eigenvalue k = 0.9122610517725336
Iteration 13: Eigenvalue k = 0.9124894190761457
Iteration 14: Eigenvalue k = 0.9126645386803544
Iteration 15: Eigenvalue k = 0.9127990586739619
Iteration 16: Eigenvalue k = 0.9129025182983138
Iteration 17: Eigenvalue k = 0.912982158439048
Iteration 18: Eigenvalue k = 0.9130435007625031
Iteration 19: Eigenvalue k = 0.9130907698523003
Iteration 20: Eigen

In [10]:
case.get_XS()

{0: [[1.4, 0.02, 0.007, [0, 0.01], 1.0], [0.4, 0.1464999, 0.2, [0, 0], 0.0]],
 1: [[1.4, 0.02, 0.007, [0, 0.01], 1.0], [0.4, 0.15, 0.2, [0, 0], 0.0]],
 2: [[1.3, 0.018000000000000002, 0.003, [0, 0.01], 1.0],
  [0.5, 0.05, 0.06, [0, 0], 0.0]]}